In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os,glob,tqdm


In [2]:
df=pd.read_excel("../data/processing_data_ot_playtime_rating.xlsx")
print(df.shape)
df.head()

(37368, 16)


,timestemp,month,day,hour,minute,dayofweek,playtime,uniquecode,productcode,productname,category,small_category,rating,price,profit,ot
0,2019-01-01 06:00:00,1,1,6,0,1,20,100346,201072,테이트 남성 셀린니트3종,의류,티셔츠/스웨터,0.0,39900,2099000,0
1,2019-01-01 06:00:00,1,1,6,0,1,20,100346,201079,테이트 여성 셀린니트3종,의류,티셔츠/스웨터,0.0,39900,4371000,0
2,2019-01-01 06:20:00,1,1,6,20,1,20,100346,201072,테이트 남성 셀린니트3종,의류,티셔츠/스웨터,0.0,39900,3262000,1
3,2019-01-01 06:20:00,1,1,6,20,1,20,100346,201079,테이트 여성 셀린니트3종,의류,티셔츠/스웨터,0.0,39900,6955000,1
4,2019-01-01 06:40:00,1,1,6,40,1,20,100346,201072,테이트 남성 셀린니트3종,의류,티셔츠/스웨터,0.0,39900,6672000,2


In [3]:
df["quantity"]=np.round(df["profit"]/df["price"])
pd.options.display.float_format = '{:.1f}'.format
df.head()

,timestemp,month,day,hour,minute,dayofweek,playtime,uniquecode,productcode,productname,category,small_category,rating,price,profit,ot,quantity
0,2019-01-01 06:00:00,1,1,6,0,1,20,100346,201072,테이트 남성 셀린니트3종,의류,티셔츠/스웨터,0.0,39900,2099000,0,53.0
1,2019-01-01 06:00:00,1,1,6,0,1,20,100346,201079,테이트 여성 셀린니트3종,의류,티셔츠/스웨터,0.0,39900,4371000,0,110.0
2,2019-01-01 06:20:00,1,1,6,20,1,20,100346,201072,테이트 남성 셀린니트3종,의류,티셔츠/스웨터,0.0,39900,3262000,1,82.0
3,2019-01-01 06:20:00,1,1,6,20,1,20,100346,201079,테이트 여성 셀린니트3종,의류,티셔츠/스웨터,0.0,39900,6955000,1,174.0
4,2019-01-01 06:40:00,1,1,6,40,1,20,100346,201072,테이트 남성 셀린니트3종,의류,티셔츠/스웨터,0.0,39900,6672000,2,167.0


In [4]:
product_name =set(df["productname"])
len(product_name)

1692

In [5]:
df["week"] =df.timestemp.dt.weekofyear

In [6]:
df.loc[(df["month"]==12) & (df["week"]==1),"week"] = 53

In [7]:
week_rank=df[["week","productname","ot"]].groupby(["week","productname"]).agg(["count","max"]).reset_index()
week_rank

week           productname    ot    
                                count max
0       1  (a4)팽현숙의 순대 곱창 전골&볶음     3   2
1       1   (도냄)무이자 쿠쿠전기밥솥 10인용     9   2
2       1    (도냄)무이자 쿠쿠전기밥솥 6인용     9   2
3       1   (도냄)일시불  쿠쿠전기밥솥 6인용     9   2
4       1   (도냄)일시불 쿠쿠전기밥솥 10인용     9   2
...   ...                   ...   ...  ..
7598   53   푸마 스트레치 심리스 드로즈 11종     3   2
7599   53     푸마 시그니처 코튼 트렁크 9종     3   2
7600   53          푸마 코튼 언더탑 9종     3   2
7601   53     피시원 국내산 햇 손질문어 7팩     3   2
7602   53       한일 대용량 스텐 분쇄믹서기     3   2

[7603 rows x 4 columns]

In [8]:
week_rank["week_rank"]=week_rank[("ot","count")]/ (week_rank[("ot","max")]+1)
week_rank

week           productname    ot     week_rank
                                count max          
0       1  (a4)팽현숙의 순대 곱창 전골&볶음     3   2       1.0
1       1   (도냄)무이자 쿠쿠전기밥솥 10인용     9   2       3.0
2       1    (도냄)무이자 쿠쿠전기밥솥 6인용     9   2       3.0
3       1   (도냄)일시불  쿠쿠전기밥솥 6인용     9   2       3.0
4       1   (도냄)일시불 쿠쿠전기밥솥 10인용     9   2       3.0
...   ...                   ...   ...  ..       ...
7598   53   푸마 스트레치 심리스 드로즈 11종     3   2       1.0
7599   53     푸마 시그니처 코튼 트렁크 9종     3   2       1.0
7600   53          푸마 코튼 언더탑 9종     3   2       1.0
7601   53     피시원 국내산 햇 손질문어 7팩     3   2       1.0
7602   53       한일 대용량 스텐 분쇄믹서기     3   2       1.0

[7603 rows x 5 columns]

In [9]:
np.max(week_rank["week_rank"])

18.0

In [10]:
np.min(week_rank["week_rank"])

0.3333333333333333

In [11]:
week_rank[week_rank["week_rank"]==18]

week         productname    ot     week_rank
                              count max          
1318   11    헤스티지 그레이스 양가죽 코트    18   0      18.0
1319   11  헤스티지 마론 양가죽 남성집업재킷    18   0      18.0

# 하루라는 개념도 다시 잡고, 주라는 개념도 다시잡자.

In [12]:
df.head(2)

,timestemp,month,day,hour,minute,dayofweek,playtime,uniquecode,productcode,productname,category,small_category,rating,price,profit,ot,quantity,week
0,2019-01-01 06:00:00,1,1,6,0,1,20,100346,201072,테이트 남성 셀린니트3종,의류,티셔츠/스웨터,0.0,39900,2099000,0,53.0,1
1,2019-01-01 06:00:00,1,1,6,0,1,20,100346,201079,테이트 여성 셀린니트3종,의류,티셔츠/스웨터,0.0,39900,4371000,0,110.0,1


In [14]:
days = df[df["month"]==1]["day"].unique()
days.sort()
days

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31],
      dtype=int64)

In [15]:
i=0
df[df["day"] == days[i]]["hour"]

0         6
1         6
2         6
3         6
4         6
         ..
34481    23
34482    23
34483    23
34484    23
34485    23
Name: hour, Length: 1237, dtype: int64

In [16]:
df.sort_values(by="timestemp",inplace=True)

# 스케줄편성으로 하루라는 개념 잡기.

In [17]:
day_list=[]
old_month = ""
old_day = ""

for i in range(df.shape[0]):
    data = df.iloc[i]
    new_month =data["month"]
    new_day =data["day"]
    new_hour =data["hour"]
    if(new_month!=old_month):
        if(new_day!=old_day):
                if(new_hour<=2):
                    day_list.append(old_day)
                else:
                    day_list.append(new_day)
                    old_day = new_day
                    old_month = new_month
        else:
            if(new_hour>=6):
                old_day=new_day
                old_month = new_month
                day_list.append(old_day)
            else:
                print("에러")
    else:
        if(new_day!=old_day):
                if(new_hour<=2):
                    day_list.append(old_day)
                else:
                    day_list.append(new_day)
                    old_day = new_day
                    old_month = new_month
        else:
            if(new_hour>=6):
                old_day=new_day
                old_month = new_month
                day_list.append(old_day)
            else:
                print("에러")

In [18]:
len(day_list)

37368

In [19]:
# 스케줄 편성된 하루 날짜.
df["schedule_day"] = day_list

In [20]:
d1=df[["month","day","hour","schedule_day"]]
d1

,month,day,hour,schedule_day
0,1,1,6,1
1,1,1,6,1
2,1,1,6,1
3,1,1,6,1
4,1,1,6,1
...,...,...,...,...
37363,12,31,23,31
37366,12,31,23,31
37364,12,31,23,31
37365,12,31,23,31


In [21]:
d1[d1["day"]!=d1["schedule_day"]]["hour"].value_counts()

1    1549
0    1492
2      70
Name: hour, dtype: int64

In [22]:
d2 = d1[d1["day"]!=d1["schedule_day"]]
d2

,month,day,hour,schedule_day
58,1,2,0,1
59,1,2,0,1
60,1,2,0,1
61,1,2,0,1
63,1,2,0,1
...,...,...,...,...
37277,12,31,1,30
37278,12,31,1,30
37279,12,31,1,30
37280,12,31,1,30


In [23]:
d2[["day","schedule_day","hour"]].groupby(["day","schedule_day"]).agg("count")

hour
day schedule_day      
1   28              12
    30              22
    31              38
2   1              130
3   2              117
4   3              109
5   4              125
6   5              124
7   6               97
8   7              148
9   8               91
10  9               94
11  10              95
12  11             104
13  12              94
14  13              99
15  14             101
16  15             103
17  16              99
18  17             104
19  18             118
20  19             100
21  20             113
22  21             101
23  22              81
24  23             119
25  24              93
26  25              93
27  26             102
28  27              77
29  28              88
30  29              61
31  30              59

In [24]:
data = df.iloc[-1]
data

timestemp         2019-12-31 23:40:00
month                              12
day                                31
hour                               23
minute                             40
dayofweek                           1
playtime                           20
uniquecode                     100448
productcode                    201391
productname             일시불쿠첸압력밥솥 6인용
category                           주방
small_category                   주방가전
rating                            0.4
price                          148000
profit                       10157000
ot                                  1
quantity                         69.0
week                               53
schedule_day                       31
Name: 37367, dtype: object

In [25]:

old_day = 1
new_day = data["schedule_day"]

# 오답 -> 스케줄 편성에 따라 스케줄 편성이란.

오전 6시부터 새벽 2시까지 방송된것에 대해서 그걸 하루 스케줄이라고 할때,  1주일동안 편성 된것을 1 2 3 4 라고 나타냄.
-- > 이걸로 인기있는 방송 카테고리 또는 트렌트를 잡아볼 예정

In [26]:
# count=0
# week=1
# week_box = []
# old_day = 1
# for i in range(df.shape[0]):
#     data = df.iloc[i]
#     new_day = data["schedule_day"]
#     if(old_day !=new_day):
#         count+=1
#         old_day = new_day
#         if(count==7):
#             count=0
#             week+=1
#         week_box.append(week)
#     else:
#         week_box.append(week)

In [27]:
# 요것이 정답. 월요일 기준으로 주를 나눈것.
count=0
week=1
week_box = []
old_day = 1
for i in range(df.shape[0]):
    data = df.iloc[i]
    new_dayofweek = data["dayofweek"]
    new_day = data["schedule_day"]
    if(i==0):
        print("처음 count",count)
        if(new_dayofweek!=0):
            count=new_dayofweek
            print("초기 count",count)
    if(old_day !=new_day):
        print(count)
        count+=1
        old_day = new_day
        if(count==7):
            print("새로운 날짜 ",old_day)
            count=0
            week+=1
            print("몇주 ",week)
        week_box.append(week)
        
    else:
        week_box.append(week)
        
       

처음 count 0
초기 count 1
1
2
3
4
5
6
새로운 날짜  7
몇주  2
0
1
2
3
4
5
6
새로운 날짜  14
몇주  3
0
1
2
3
4
5
6
새로운 날짜  21
몇주  4
0
1
2
3
4
5
6
새로운 날짜  28
몇주  5
0
1
2
3
4
5
6
새로운 날짜  4
몇주  6
0
1
2
3
4
5
6
새로운 날짜  11
몇주  7
0
1
2
3
4
5
6
새로운 날짜  18
몇주  8
0
1
2
3
4
5
6
새로운 날짜  25
몇주  9
0
1
2
3
4
5
6
새로운 날짜  4
몇주  10
0
1
2
3
4
5
6
새로운 날짜  11
몇주  11
0
1
2
3
4
5
6
새로운 날짜  18
몇주  12
0
1
2
3
4
5
6
새로운 날짜  25
몇주  13
0
1
2
3
4
5
6
새로운 날짜  1
몇주  14
0
1
2
3
4
5
6
새로운 날짜  8
몇주  15
0
1
2
3
4
5
6
새로운 날짜  15
몇주  16
0
1
2
3
4
5
6
새로운 날짜  22
몇주  17
0
1
2
3
4
5
6
새로운 날짜  29
몇주  18
0
1
2
3
4
5
6
새로운 날짜  6
몇주  19
0
1
2
3
4
5
6
새로운 날짜  13
몇주  20
0
1
2
3
4
5
6
새로운 날짜  20
몇주  21
0
1
2
3
4
5
6
새로운 날짜  27
몇주  22
0
1
2
3
4
5
6
새로운 날짜  3
몇주  23
0
1
2
3
4
5
6
새로운 날짜  10
몇주  24
0
1
2
3
4
5
6
새로운 날짜  17
몇주  25
0
1
2
3
4
5
6
새로운 날짜  24
몇주  26
0
1
2
3
4
5
6
새로운 날짜  1
몇주  27
0
1
2
3
4
5
6
새로운 날짜  8
몇주  28
0
1
2
3
4
5
6
새로운 날짜  15
몇주  29
0
1
2
3
4
5
6
새로운 날짜  22
몇주  30
0
1
2
3
4
5
6
새로운 날짜  29
몇주  31
0
1
2
3
4
5
6
새로운 날짜  5
몇주  32
0
1
2


In [28]:
max(week_box)

53

In [29]:
df["schedule_week"] = week_box

In [31]:
df[df["week"] !=df["schedule_week"]]["hour"].value_counts()

0    280
1    212
2      7
Name: hour, dtype: int64

In [32]:
a1=df[["schedule_day","week","schedule_week","hour"]]
a1

,schedule_day,week,schedule_week,hour
0,1,1,1,6
1,1,1,1,6
2,1,1,1,6
3,1,1,1,6
4,1,1,1,6
...,...,...,...,...
37363,31,53,53,23
37366,31,53,53,23
37364,31,53,53,23
37365,31,53,53,23


In [33]:
a1[a1["week"]!=a1["schedule_week"]].groupby(["schedule_week","hour"]).agg("count").head(20)

schedule_day  week
schedule_week hour                    
1             0                2     2
              1                6     6
2             0               12    12
              1                3     3
3             0                3     3
              1                3     3
4             0                3     3
              1                3     3
5             0                3     3
              1                3     3
6             0                3     3
              1                6     6
7             0                3     3
              1                6     6
8             0                3     3
              1                9     9
9             0                9     9
              1                3     3
10            0                5     5
              1                6     6

In [34]:
d2[["day","schedule_day","hour"]].groupby(["day","schedule_day"]).agg("count")

hour
day schedule_day      
1   28              12
    30              22
    31              38
2   1              130
3   2              117
4   3              109
5   4              125
6   5              124
7   6               97
8   7              148
9   8               91
10  9               94
11  10              95
12  11             104
13  12              94
14  13              99
15  14             101
16  15             103
17  16              99
18  17             104
19  18             118
20  19             100
21  20             113
22  21             101
23  22              81
24  23             119
25  24              93
26  25              93
27  26             102
28  27              77
29  28              88
30  29              61
31  30              59

In [35]:
min(week_box)

1

In [36]:
df = df[['timestemp', 'month', 'day', 'hour', 'minute', 'dayofweek','week',
       'schedule_day', 'schedule_week', 'playtime',
       'uniquecode', 'productcode', 'productname', 'category',
       'small_category', 'rating', 'price', 'profit', 'ot', 'quantity']]

In [37]:
# 파일 저장.
#df.to_excel("../data/processing_data_ot_playtime_rating_schedule.xlsx",index=False)

In [38]:
df.tail(50)

,timestemp,month,day,hour,minute,dayofweek,week,schedule_day,schedule_week,playtime,uniquecode,productcode,productname,category,small_category,rating,price,profit,ot,quantity
37318,2019-12-31 15:20:00,12,31,15,20,1,53,31,53,20,100812,202397,제주갈치+참조기세트,농수축,생선류,0.3,49900,34001000,0,681.0
37319,2019-12-31 15:40:00,12,31,15,40,1,53,31,53,20,100812,202397,제주갈치+참조기세트,농수축,생선류,0.3,49900,54234000,1,1087.0
37320,2019-12-31 16:00:00,12,31,16,0,1,53,31,53,20,100812,202397,제주갈치+참조기세트,농수축,생선류,0.0,49900,52313000,2,1048.0
37322,2019-12-31 16:20:00,12,31,16,20,1,53,31,53,20,100231,200794,무이자 키친플라워 대용량 에어프라이어 5.8L,주방,주방가전,0.0,74900,5329000,0,71.0
37321,2019-12-31 16:20:00,12,31,16,20,1,53,31,53,20,100231,200808,일시불 키친플라워 대용량 에어프라이어 5.8L,주방,주방가전,0.0,70900,15573000,0,220.0
37323,2019-12-31 16:40:00,12,31,16,40,1,53,31,53,20,100231,200808,일시불 키친플라워 대용량 에어프라이어 5.8L,주방,주방가전,0.3,70900,20228000,1,285.0
37324,2019-12-31 16:40:00,12,31,16,40,1,53,31,53,20,100231,200794,무이자 키친플라워 대용량 에어프라이어 5.8L,주방,주방가전,0.3,74900,4465000,1,60.0
37325,2019-12-31 17:00:00,12,31,17,0,1,53,31,53,20,100231,200808,일시불 키친플라워 대용량 에어프라이어 5.8L,주방,주방가전,0.1,70900,37062000,2,523.0
37326,2019-12-31 17:00:00,12,31,17,0,1,53,31,53,20,100231,200794,무이자 키친플라워 대용량 에어프라이어 5.8L,주방,주방가전,0.1,74900,14411000,2,192.0
37327,2019-12-31 17:20:00,12,31,17,20,1,53,31,53,20,100236,200806,벨라홈 스마트 멀티포트 1+1 세트,주방,주방가전,0.0,39800,35005000,0,880.0


In [39]:
df

,timestemp,month,day,hour,minute,dayofweek,week,schedule_day,schedule_week,playtime,uniquecode,productcode,productname,category,small_category,rating,price,profit,ot,quantity
0,2019-01-01 06:00:00,1,1,6,0,1,1,1,1,20,100346,201072,테이트 남성 셀린니트3종,의류,티셔츠/스웨터,0.0,39900,2099000,0,53.0
1,2019-01-01 06:00:00,1,1,6,0,1,1,1,1,20,100346,201079,테이트 여성 셀린니트3종,의류,티셔츠/스웨터,0.0,39900,4371000,0,110.0
2,2019-01-01 06:20:00,1,1,6,20,1,1,1,1,20,100346,201072,테이트 남성 셀린니트3종,의류,티셔츠/스웨터,0.0,39900,3262000,1,82.0
3,2019-01-01 06:20:00,1,1,6,20,1,1,1,1,20,100346,201079,테이트 여성 셀린니트3종,의류,티셔츠/스웨터,0.0,39900,6955000,1,174.0
4,2019-01-01 06:40:00,1,1,6,40,1,1,1,1,20,100346,201072,테이트 남성 셀린니트3종,의류,티셔츠/스웨터,0.0,39900,6672000,2,167.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37363,2019-12-31 23:20:00,12,31,23,20,1,53,31,53,20,100448,201391,일시불쿠첸압력밥솥 6인용,주방,주방가전,0.0,148000,1664000,0,11.0
37366,2019-12-31 23:40:00,12,31,23,40,1,53,31,53,20,100448,201384,무이자쿠첸압력밥솥 6인용,주방,주방가전,0.4,158000,2328000,1,15.0
37364,2019-12-31 23:40:00,12,31,23,40,1,53,31,53,20,100448,201383,무이자쿠첸압력밥솥 10인용,주방,주방가전,0.4,178000,9149000,1,51.0
37365,2019-12-31 23:40:00,12,31,23,40,1,53,31,53,20,100448,201390,일시불쿠첸압력밥솥 10인용,주방,주방가전,0.4,168000,15282000,1,91.0
